In [1]:
from risk_targeted_hazard import *

In [2]:
folder = 'data'

oq_id = [int(re.split('calc_|\.',name)[1]) for name in glob.glob(str(Path(folder,'calc_*.hdf5')))][0]
file_id = str(Path(folder,f'calc_{oq_id}.hdf5'))

### retrieve hcurves from oq file

In [3]:
data = retrieve_data(file_id)

### calculate APoE based design intensities

In [4]:
hazard_rps = np.array([25,50,500,2500])
[im_hazard, stats_im_hazard] = calculate_hazard_design_intensities(data,hazard_rps)

data['hazard_design'] = {}
data['hazard_design']['hazard_rps'] = hazard_rps.tolist()
data['hazard_design']['im_hazard'] = im_hazard.tolist()
data['hazard_design']['stats_im_hazard'] = stats_im_hazard.tolist()

C:\Users\ahul697\Desktop\Research\GitHub_Repos\GNS\risk-targeted-hazard\risk_targeted_hazard\prepare_design_intensities.py:30: RuntimeWarning: divide by zero encountered in log
  im_hazard[i_site,i_imt,:,i_rlz,0] = np.exp(np.interp(np.log(1/hazard_rps), np.log(np.flip(hcurves_rlzs[i_site,i_imt,:,i_rlz])), np.log(np.flip(imtls[imt]))))
C:\Users\ahul697\Desktop\Research\GitHub_Repos\GNS\risk-targeted-hazard\risk_targeted_hazard\prepare_design_intensities.py:42: RuntimeWarning: divide by zero encountered in log
  stats_im_hazard[i_site,i_imt,:,i_stat] = np.exp(np.interp(np.log(1/hazard_rps), np.log(np.flip(hcurves_stats[i_site,i_imt,:,i_stat])), np.log(np.flip(imtls[imt]))))


### calculate risk-targeted design intensities

In [5]:
risk_target = 1e-5
beta = 0.6

risk_assumptions = {}

risk_assumptions['Horspool'] = {'risk_target':risk_target,
                                'conditional_prob':10**-4,
                                'beta':beta}

risk_assumptions['Douglas'] = {'risk_target':risk_target,
                                'conditional_prob':10**-5,
                                'beta':beta}

risk_assumptions['Silva'] = {'risk_target':risk_target,
                                'conditional_prob':10**-3,
                                'beta':beta}

In [7]:
[im_risk, stats_im_risk] = calculate_risk_design_intensities(data,risk_assumptions)

data['risk_design'] = {}
data['risk_design']['risk_assumptions'] = risk_assumptions
data['risk_design']['im_risk'] = im_risk
data['risk_design']['stats_im_risk'] = stats_im_risk

In [17]:
filename = str(Path(folder,f'design_{oq_id}.json'))
with open(filename, 'w') as f:
    json.dump(data,f,indent=4)